In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Збір даних

Перш за все, потрібно зберемо дані з акселерометра мобільного телефону.

![Alt text](image.png)

In [2]:
from pathlib import Path
activities = ['idle', 'running', 'stairs', 'walking']
data_collections = []

for index, activity in enumerate(activities, start=1):
    activity_path = Path('data') / activity  
    csv_files = list(activity_path.glob('*.csv'))  
    df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
    df['activity'] = activity
    data_collections.append(df)

data = pd.concat(data_collections, ignore_index=True)
data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking
193856,6.373365,-11.741165,-8.226476,walking
193857,3.289633,-9.993398,-0.383072,walking
193858,-2.978387,-3.050213,1.273715,walking


# Тренування на вихідних нормалізованих данних

В якості характеристик візьмемо показники з акселерометра

In [3]:
features = ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']

## Нормалізація даних

Тренування на ненормаліованих даних займає чимало часу. Вхідні дані нормалізуються так, щоб їхнє середнє значення дорівнювало нулю, а стандартне відхилення - одиниці. 

In [4]:
scaler = StandardScaler()

data_to_normalize = data[features]

# Нормалізація даних
normalized_data = scaler.fit_transform(data_to_normalize)

# Заміна вихідних даних нормалізованими даними
ndata = data.copy()
ndata[features] = normalized_data
ndata

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.109791,0.241917,0.941538,idle
1,-0.143404,0.209288,0.923560,idle
2,-0.337109,-0.150782,1.028762,idle
3,0.377889,-0.116234,0.919565,idle
4,-0.019205,-0.141185,1.035420,idle
...,...,...,...,...
193855,0.379029,-1.366884,-0.455385,walking
193856,0.529435,-1.069384,-1.394879,walking
193857,0.162535,-0.929271,-0.304241,walking
193858,-0.583227,-0.372659,-0.073862,walking


## Розділяємо дані

Розділяємо датасет на навчальний і тестовий набори

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data[features], data["activity"],
    test_size=0.3,
    stratify=ndata["activity"],
)

## Навчаємо за допомогою алгоритму SVM

In [6]:
%%time

model_svm = SVC()
model_svm.fit(X_train, y_train)

CPU times: total: 3min 36s
Wall time: 3min 51s


SVC()

## Навчаємо за допомогою алгоритму випадкового лісу

In [7]:
%%time

model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)

CPU times: total: 13 s
Wall time: 14.6 s


RandomForestClassifier()

## Порівнюємо точність результатів

Визначаємо точність моделей на тестовому наборі та порівнюємо

In [8]:
score_svm = model_svm.score(X_test, y_test)
score_rf = model_rf.score(X_test, y_test)

print(f"Точність моделі SVM: {score_svm:.3f}")
print(f"Точність моделі випадкового лісу: {score_rf:.3f}")

Точність моделі SVM: 0.892
Точність моделі випадкового лісу: 0.999


# Тренування на підготовленних даних

## Вибираємо ознаки

В статті пропонується обрати наступні ознаки (тпблиця 3)

| Name                          | Axis |
| ----------------------------- | ---- |
| Maximum Value                 |   x  |
| Minimum Value                 |   x  |
| Entropy                       |   x  |
| Interquartile Range           |   x  |
| Maximum Value                 |   y  |
| Index of Minimum Value        |   y  |
| Mean of Absolute Deviation    |   y  |
| Median                        |   y  |
| Skewness                      |   y  |
| Standard Deviation            |   y  |
| Root Mean Square Error        |   y  |
| Skewness                      |   z  |

In [9]:
data_tdf = ndata.copy()

def iqr(data):
    sorted_data = np.sort(data)
    
    Q1 = np.percentile(sorted_data, 25)
    Q3 = np.percentile(sorted_data, 75)
    
    IQR = Q3 - Q1
    
    return IQR
  
def argmin(data):

  return np.argmin(data)

def entropy(column_data):

    hist, bins = np.histogram(column_data, bins='auto')
    probs = hist / len(column_data)

    probs = probs[probs > 0]

    entropy = -np.sum(probs * np.log2(probs))

    return entropy

def mad(data):

    mean = np.mean(data)
    
    # Вычисляем абсолютные отклонения от среднего значения
    absolute_deviations = np.abs(data - mean)
    
    # Вычисляем MAD как среднее абсолютных отклонений
    mad = np.mean(absolute_deviations)
    
    return mad

def rmse(data):

    mean = np.mean(data)
    
    squared_errors = [(x - mean) ** 2 for x in data]
    
    rmse = np.sqrt(np.mean(squared_errors))
    
    return rmse

stat_features = ['max', 'min', entropy, iqr, argmin, mad, 'median', 'skew', 'std', rmse]

data_tdf = data.groupby(['activity', data.index // 30]).agg(
    stat_features
).reset_index(level=0)


In [10]:
data_tdf

activity accelerometer_X                                                 \
                          max       min   entropy       iqr argmin       mad   
0        idle        5.099650 -0.909797  3.075701  0.065840      2  0.539706   
1        idle        0.407014 -0.320823  2.732915  0.039505      3  0.068251   
2        idle       -0.062249 -0.124498  2.389898  0.021548      7  0.011982   
3        idle        0.560243  0.268151  2.663458  0.050279     29  0.037903   
4        idle        0.560243  0.268151  2.384778  0.051476     27  0.038627   
...       ...             ...       ...       ...       ...    ...       ...   
6457  walking        7.192182 -5.990292  2.507218  5.601235      1  2.911828   
6458  walking        7.192182 -7.359776  2.421421  5.035007     29  2.890440   
6459  walking       11.113884 -7.359776  2.295601  5.007473     27  3.107503   
6460  walking       11.113884 -7.359776  2.441946  4.416105     25  3.127189   
6461  walking       11.113884 -7.359776  2.380542  6.479907     23  3.422942   

                                    ... accelerometer_Z                       \
        median      skew       std  ...             max        min   entropy   
0    -0.102950  4.044308  1.036361  ...        9.806650   8.418014  2.277957   
1    -0.114922  2.300676  0.125848  ...        9.806650   9.667787  2.485918   
2    -0.100556  0.604258  0.015642  ...        9.792285   9.739613  2.526027   
3     0.392649  0.401492  0.052889  ...        9.797073   9.725247  2.333057   
4     0.402226  0.387552  0.053187  ...        9.797073   9.725247  2.357544   
...        ...       ...       ...  ...             ...        ...       ...   
6457  0.390255 -0.053479  3.474121  ...       32.599450 -20.058622  2.757882   
6458  0.390255 -0.224909  3.533930  ...       32.599450 -20.058622  2.811537   
6459  1.501165  0.124943  3.948034  ...       32.599450 -16.903065  2.755913   
6460  1.889025 -0.099428  4.016935  ...       32.599450 -16.903065  2.850050   
6461  1.501165  0.084357  4.212427  ...       32.599450 -16.903065  2.771766   

                                                                         
           iqr argmin       mad    median      skew       std      rmse  
0     0.034716      3  0.275546  9.770737 -2.365312  0.406903  0.400064  
1     0.019153     10  0.016664  9.777920 -2.409922  0.025419  0.024992  
2     0.019154     29  0.011673  9.768343 -0.097477  0.014528  0.014284  
3     0.022745     20  0.013886  9.751583  0.565957  0.017967  0.017665  
4     0.027533     18  0.015004  9.751583  0.453139  0.018902  0.018584  
...        ...    ...       ...       ...       ...       ...       ...  
6457  2.989161      2  4.750405  0.833182  1.221757  8.666662  8.520994  
6458  2.989161      0  4.754246  0.847548  1.178148  8.675810  8.529988  
6459  2.989161     14  4.165187  0.833182  1.996085  7.863650  7.731478  
6460  2.989161     12  4.260509  0.768539  1.959435  7.943439  7.809926  
6461  2.980781     10  4.260296  0.768539  1.950320  7.941497  7.808017  

[6462 rows x 31 columns]

In [11]:
data_tdf.drop(
    [
        ("accelerometer_X", "argmin"),
        ("accelerometer_X", "mad"),
        ("accelerometer_X", "median"),
        ("accelerometer_X", "skew"),
        ("accelerometer_X", "std"),
        ("accelerometer_X", "rmse"),
        ("accelerometer_Y", "iqr"),
        ("accelerometer_Y", "min"),
        ("accelerometer_Y", "entropy"),
        ("accelerometer_Y", "iqr"),
        ("accelerometer_Z", "max"),
        ("accelerometer_Z", "min"),
        ("accelerometer_Z", "entropy"),
        ("accelerometer_Z", "iqr"),
        ("accelerometer_Z", "argmin"),
        ("accelerometer_Z", "mad"),
        ("accelerometer_Z", "median"),
        ("accelerometer_Z", "std"),
        ("accelerometer_Z", "rmse")
    ],
    axis=1,
    inplace=True
)


## Розділяємо датасет на навчальний і тестовий набори

In [12]:

X_train, X_test, y_train, y_test = train_test_split(
    data_tdf[features], data_tdf["activity"],
    test_size=0.3,
    stratify=data_tdf["activity"],
)

## Навчаємо за домомогою алгоритму `SVN`

In [13]:
%%time

model_svm = SVC()
model_svm.fit(X_train, y_train)

CPU times: total: 62.5 ms
Wall time: 76 ms


SVC()

## Навчаємо за домомогою алгоритму `random forest`

In [14]:
%%time

model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)

CPU times: total: 812 ms
Wall time: 959 ms


RandomForestClassifier()

## Визначаємо точність моделей на тестовому наборі

In [15]:
score_svm = model_svm.score(X_test, y_test)
score_rf = model_rf.score(X_test, y_test)

print(f"Точність моделі SVM: {score_svm:.3f}")
print(f"Точність моделі випадкового лісу: {score_rf:.3f}")

Точність моделі SVM: 0.975
Точність моделі випадкового лісу: 0.997


# Висновки 


1. Для навчання алгоритму `SVN` на __вихідних ознаках__ (показах акселерометра по трьох осях) потрібно близько 20 хвилин на стандартному комп'ютері. Навчання на вихідних ознаках (показах акселерометра по трьох осях) спричиняє довге навчання, оскільки алгоритм `SVN` має велику кількість параметрів, які потрібно оптимізувати. Це пов'язано з тим, що алгоритм `SVN` намагається знайти оптимальну підмножину ознак, яка найкраще класифікує дані. У випадку з даними про рух людини, які мають багато шуму і невизначеності, це може бути дуже складним завданням.

2. Навчання на __нормалізованих ознаках__ акселерометра по трьох осях зменшує час навчання алгоритму `SVN` до близько 4 хвилин. Навчання на нормалізованих показах акселерометра по трьох осях дає значний виграш при навчання по методу `SVN`, оскільки це зменшує кількість параметрів, які потрібно оптимізувати. Нормування показів акселерометра по трьох осях усуває масштабування, яке може призвести до переваги одних ознак над іншими. Це також робить дані більш однорідними, що полегшує задачу навчання алгоритму `SVN`.
    - час розрахунку близько 4 хвилин;
    - точність 0.892.

3. Навчання за методом випадкового лісу показує кращу точність та набагато менший час навчання, оскільки це більш стійкий до шуму і невизначеності алгоритм. Метод випадкового лісу генерує декілька дерев рішень, і кожен з них голосує за класифікацію. Це дозволяє алгоритму випадкового лісу бути більш точним, ніж алгоритм `SVN`, який генерує лише одне дерево рішень. Крім того, метод випадкового лісу має набагато меншу кількість параметрів, ніж алгоритм `SVN`, що також сприяє його більш швидкому навчанню.
    - час розрахунку близько 13 секунд;
    - точність 0.999.

4. Навчання, де в якості ознак взяті статистичні ознаки, отримані по набору з 30 показників акселерометра для кожної осі $x$, $y$ та $z$ показує:
    - точність моделі `SVM` значно підвищилась до значення 0.975, крім того час навчання значно зменшився і становить 46.9 ms.
    - точність моделі `random forest` дещо знизилась в порівнянні з навчанням на вихідних нормалізованих даних до величини 0.994, час навчання становить 766 ms, що значно менше ніж для навчання на вихідних нормалізованих даних, але натомість він значно вище в порявнянні з моделлю `SVN`.

Результати наведено в таблиці

<table>
<thead>
  <tr>
    <th>Тип ознак</th>
    <th>Алгоритм</th>
    <th>Час розрахунку</th>
    <th>Точність</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="2">вихідні нормалізовані</td>
    <td>SVN</td>
    <td>240 s</td>
    <td>0.892</td>
  </tr>
  <tr>
    <td>Random Forest</td>
    <td>13 s</td>
    <td>0.999</td>
  </tr>
  <tr>
    <td rowspan="2">статистичні дані</td>
    <td>SVN</td>
    <td>46.9 ms</td>
    <td>0.975</td>
  </tr>
  <tr>
    <td>Random Forest</td>
    <td>766 ms</td>
    <td>0.994</td>
  </tr>
</tbody>
</table>

Згідно таблиці, можна зробити висновок, що найкращі результати дає в сенсі час/чточність дає алгоритм `random forest`, який тренувався не на вихідних, а на на статистичних ознаках.